In [12]:
import twitter
import pandas as pd

In [13]:
consumer_key = '3hadC5cGfmdfYqVQ3EClF0VZd'
secret_key ='kjIfrQrSokxLirRZ21HstrJOJQZvuM9FmUDkNpRYrYjmhLtyWK'

access_token = '374743137-e4C4fLpqDqfjv7dVpe5S9SFQbWKZ9Ii8ci3dFa53'
secret_token = 'tGsYrurk2euQwZwaf2vZeuJCBQNTTYl7tiPmF7Qqffq2S'

In [15]:
from twitter import *
t = twitter.api.Twitter(auth=OAuth(access_token , secret_token, consumer_key, secret_key))
t.statuses.home_timeline()[0]

{'created_at': 'Sat Apr 04 17:25:42 +0000 2020',
 'id': 1246489134006493185,
 'id_str': '1246489134006493185',
 'text': 'Here are 24 things I want you to know about @TheKalenAllen. https://t.co/4LrekC9YBA',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'TheKalenAllen',
    'name': 'Kalen Allen',
    'id': 171685697,
    'id_str': '171685697',
    'indices': [44, 58]}],
  'urls': [],
  'media': [{'id': 1246488139440553992,
    'id_str': '1246488139440553992',
    'indices': [60, 83],
    'media_url': 'http://pbs.twimg.com/amplify_video_thumb/1246488139440553992/img/o2qOtiXpXtNK7u4v.jpg',
    'media_url_https': 'https://pbs.twimg.com/amplify_video_thumb/1246488139440553992/img/o2qOtiXpXtNK7u4v.jpg',
    'url': 'https://t.co/4LrekC9YBA',
    'display_url': 'pic.twitter.com/4LrekC9YBA',
    'expanded_url': 'https://twitter.com/TheEllenShow/status/1246489134006493185/video/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h

In [16]:
#Query Params
q=Any Text
geocode
lang
result_type=recent, popular, mixed
until=anything before date in format YYYY-MM-DD good for one week onl
since_id=only return tweets after a given tweet
max_id = returns tweets only before a given date

Help on Twitter in module twitter.api object:

class Twitter(TwitterCall)
 |  Twitter(format='json', domain='api.twitter.com', secure=True, auth=None, api_version=<class 'twitter.api._DEFAULT'>, retry=False)
 |  
 |  The minimalist yet fully featured Twitter API class.
 |  
 |  Get RESTful data by accessing members of this class. The result
 |  is decoded python objects (lists and dicts).
 |  
 |  The Twitter API is documented at:
 |  
 |    https://dev.twitter.com/overview/documentation
 |  
 |  The list of most accessible functions is listed at:
 |  
 |    https://dev.twitter.com/rest/public
 |  
 |  
 |  Examples::
 |  
 |      from twitter import *
 |  
 |      t = Twitter(
 |          auth=OAuth(token, token_secret, consumer_key, consumer_secret))
 |  
 |      # Get your "home" timeline
 |      t.statuses.home_timeline()
 |  
 |      # Get a particular friend's timeline
 |      t.statuses.user_timeline(screen_name="billybob")
 |  
 |      # to pass in GET/POST parameters, such as 

In [113]:
#t.search.tweets(q='#covid')
resp = t.search.tweets(geocode='39.0146,-77.03705,1mi',count=1000, result_type='recent')
resp['search_metadata']

{'completed_in': 0.12,
 'max_id': 1246518490179276800,
 'max_id_str': '1246518490179276800',
 'next_results': '?max_id=1245112747035185153&q=&geocode=39.0146%2C-77.03705%2C1mi&count=100&include_entities=1&result_type=recent',
 'query': '',
 'refresh_url': '?since_id=1246518490179276800&q=&geocode=39.0146%2C-77.03705%2C1mi&result_type=recent&include_entities=1',
 'count': 100,
 'since_id': 0,
 'since_id_str': '0'}

In [199]:
#other stuff that can be pulled, user mentions and the details of those users-including a member of the house of representatives.  
#retweet rate, favorites etc
#pull by user:  followers, following, tweet history etc.  

def format_tweets(resp):
    my_list = []
    counter = 0
    for i in resp['statuses']:
        date = i['created_at']
        tweet_id = i['id']
        text = i['text']
        user_id = i['user']['id']
        user_name = i['user']['name']
        user_description=i['user']['description']
        user_location = i['user']['location']
        geo = i['geo']
        coordinates = i['coordinates']
        place_name = i['place']['full_name']
        hashtag = i['entities']['hashtags']
        my_list.append([date, text, user_id, user_name, user_description, user_location, geo, coordinates, place_name,hashtag,tweet_id])
    tweet_data = pd.DataFrame(my_list, columns =['date', 'text', 'user_id', 'user_name', 'user_description', 'user_location', 'geo', 'coordinates', 'place_name','hashtag','tweet_id'])
    tweet_data.date = pd.to_datetime(tweet_data.date)
    max_id = tweet_data.loc[tweet_data.date == min(tweet_data.date),'tweet_id'].values
    return(max_id, tweet_data)

In [200]:
last_id, tweet_data = format_tweets(resp)

In [201]:
resp2 = t.search.tweets(geocode='39.0146,-77.03705,1mi',count=1000, result_type='recent',max_id=last_id[0])

In [202]:
last_id2, tweet_data2 = format_tweets(resp2)

In [206]:
tweet_data2

,date,text,user_id,user_name,user_description,user_location,geo,coordinates,place_name,hashtag,tweet_id
0,2020-03-31 22:16:26,"Rick Fox, Kwame Brown, Robert Sacre 😭 no offen...",776291142,Ty 🧩,Spirit-Filled. Husband. Servant. Student. | ig...,"Washington, DC",None,None,"Forest Glen, MD",[],1245112747035185154
1,2020-03-31 18:12:53,@DeclanMcKenna do guitar string,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245051454123118592
2,2020-03-31 17:13:17,Some rando with 7 followers tried coming for m...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245036458207064069
3,2020-03-31 17:10:37,@DeclanMcKenna will you be in lice gangs full ...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245035786145353731
4,2020-03-31 17:09:04,@DeclanMcKenna will you ever cover Bedroom Flo...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245035393747165184
5,2020-03-31 17:07:47,Why is d*clan McKenna being active right now I...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245035071226216450
6,2020-03-31 17:03:51,Who wants to pitch in so the Declan fandom can...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245034082926497792
7,2020-03-31 16:56:22,I’m literally so excited to wear this to my on...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1245032198228643840
8,2020-03-31 16:26:38,@NorthCaroliNats I just can’t even. Speechless...,1135819927,Michigan Girl ready for ⚾️,Nats ⚾️ Caps 🏒 University of Michigan 〽️💛💙 #HS...,,None,None,"Forest Glen, MD",[],1245024717045710853
9,2020-03-31 13:56:06,I am so here for @ChappellRoan turning her bra...,2298103462,Emma💕 ☼,🤠 was @accioselfesteem,"Silver Spring, MD",None,None,"Forest Glen, MD",[],1244986831449587712


In [204]:
max(tweet_data.date)

Timestamp('2020-04-04 19:22:21')